# Gemini 3 Script Generator with Knowledge Graph

This notebook demonstrates how to use the Gemini 3 API (via `google-genai`) to generate scripts based on user input, enriched with context from a Neo4j Knowledge Graph.

## Prerequisites
- Neo4j Database running (with the Fabric Knowledge Graph loaded)
- Google Gemini API Key


In [1]:
# Install dependencies
%pip install neo4j google-genai sentence-transformers python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import numpy as np
from dotenv import load_dotenv, find_dotenv
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google import genai
from google.genai import types

# Load environment variables
# Try to find .env file explicitly if not found automatically
dotenv_path = find_dotenv()
if dotenv_path:
    print(f"Loading .env from: {dotenv_path}")
    load_dotenv(dotenv_path, override=True)
else:
    print("⚠️ .env file not found. Please ensure it exists in the project root.")
    # Fallback: try loading from current directory
    load_dotenv(override=True)

print("Libraries imported successfully.")

Loading .env from: /Users/shreyasjagannath/dev/fabric/.env
Libraries imported successfully.


In [3]:
# Neo4j Configuration
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = None
        try:
            self.driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
            with self.driver.session() as session:
                session.run("RETURN 1")
                print("✓ Successfully connected to Neo4j")
        except Exception as e:
            print(f"✗ Connection failed: {e}")
            
        
    def close(self):
        if self.driver:
            self.driver.close()
    
    def query(self, query_str, **kwargs):
        with self.driver.session() as session:
            return session.run(query_str, **kwargs).data()

# Initialize connection
neo4j_conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

✓ Successfully connected to Neo4j


In [4]:
# Initialize Gemini Client
# Ensure you have GOOGLE_API_KEY or GEMINI_API_KEY in your .env file
api_key = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")

if not api_key:
    print("⚠️ GOOGLE_API_KEY or GEMINI_API_KEY not found in environment variables.")
    print("Gemini features will be disabled until a valid key is provided.")
    client = None
else:
    client = genai.Client(api_key=api_key)
    print("Gemini client initialized.")

Gemini client initialized.


In [29]:
# Initialize Embedding Model
print("Loading embedding model (this may take a moment)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Embedding model loaded.")

def get_embedding(text):
    return embedding_model.encode(text)

def search_knowledge_graph(query_text, limit=5):
    """
    Search the knowledge graph for relevant Topics, Entities and Categories using semantic similarity.
    """
    if not neo4j_conn:
        return []
        
    query_embedding = get_embedding(query_text)
    
    # Search for Topics, Entities and Categories
    cypher_query = f"""
    MATCH (n)
    WHERE (n:Topic OR n:Entity) AND n.embedding IS NOT NULL
    RETURN 
        labels(n)[0] as node_type,
        coalesce(n.name, n.content) as content,
        n.embedding as embedding
    LIMIT {limit * 5}
    """
    
    results = neo4j_conn.query(cypher_query)
    
    similarities = []
    for result in results:
        node_embedding = np.array(result['embedding'])
        query_vec = np.array(query_embedding).reshape(1, -1)
        node_vec = node_embedding.reshape(1, -1)
        similarity = cosine_similarity(query_vec, node_vec)[0][0]
        
        similarities.append({
            'type': result['node_type'],
            'content': result['content'],
            'similarity': float(similarity)
        })
    
    # Sort by similarity and take top N
    similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)
    return similarities[:limit]

Loading embedding model (this may take a moment)...
✓ Embedding model loaded.
✓ Embedding model loaded.


In [30]:
def generate_script_with_context(user_request):
    """
    Generates a video/movie script based on user request, using Knowledge Graph context.
    """
    print(f"🔍 Analyzing request: '{user_request}'")
    
    # 1. Get Context from Knowledge Graph
    print("   Querying Knowledge Graph...")
    kg_results = search_knowledge_graph(user_request, limit=10)
    
    context_str = ""
    if kg_results:
        print("\n   --- Relevant Knowledge Graph Items ---")
        context_str = "Relevant Context from Knowledge Graph (User Interests/Topics):\n"
        for item in kg_results:
            item_str = f"- [{item['type']}] {item['content']} (Relevance: {item['similarity']:.2f})"
            print(f"   {item_str}")
            context_str += item_str + "\n"
        print("   --------------------------------------\n")
    else:
        context_str = "No specific context found in Knowledge Graph."
        
    print(f"   Found {len(kg_results)} relevant items.")
    
    # 2. Construct Prompt for Gemini
    prompt = f"""
    You are an expert creative writer and video scriptwriter.
    
    User Request: {user_request}
    
    {context_str}
    
    Task: Create a creative video/movie script that addresses the user's request.
    
    Guidelines:
    1. Use the provided context to personalize the script to the user's specific interests found in the knowledge graph.
    2. Structure the script clearly with:
       - **Title**
       - **Scene/Setting Description**
       - **Characters** (if applicable)
       - **Dialogue/Voiceover** (with time markers if possible)
       - **Visual Cues/Action**
    3. Keep the tone aligned with the user's request (e.g., fun, professional, dramatic).
    4. Keep the script concise but engaging, ideally between 300-500 words and should be a movie/video of 8 seconds.
    
    Output the script in a clear, readable format (Markdown).
    """
    
    # 3. Call Gemini API
    print("   Generating script with Gemini...")
    
    if client is None:
        return "❌ Gemini Client not initialized. Please provide GOOGLE_API_KEY."

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash-exp", # Or gemini-1.5-flash
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"Error generating content: {e}"

print("Generator function defined.")

Generator function defined.


In [31]:
# # Example Usage
# user_request = "Can you generate a video about london"
# script = generate_script_with_context(user_request)

# print("\n" + "="*40 + "\nGENERATED SCRIPT\n" + "="*40 + "\n")
# print(script)

In [32]:
import time
import requests
import os
import datetime

def create_video_from_script(script_text):
    """
    Extracts a visual prompt from the script and generates a video using Veo 3.
    """
    if not script_text:
        print("❌ No script provided.")
        return

    # 1. Extract Visual Prompt using Gemini
    print("🎨 Extracting visual prompt from script...")
    extraction_prompt = f"""
    Based on the following video script, create a concise, high-quality visual prompt for an AI video generator (like Veo).
    Focus ONLY on the visual elements, setting, lighting, and action. 
    Do not include dialogue or sound instructions.
    The video is 8 seconds long.
    
    Script:
    {script_text}
    
    Output the visual prompt only.
    """
    
    try:
        visual_response = client.models.generate_content(
            model="gemini-2.0-flash-exp",
            contents=extraction_prompt
        )
        visual_prompt = visual_response.text.strip()
        print(f"   Visual Prompt: '{visual_prompt}'")
    except Exception as e:
        print(f"❌ Error extracting visual prompt: {e}")
        return

    # 2. Generate Video with Veo 3
    print(f"\n🎥 Generating video with Veo...")
    
    # Initialize v1alpha client for Veo
    try:
        from google.genai.types import HttpOptions
        alpha_client = genai.Client(
            api_key=api_key,
            http_options=HttpOptions(api_version='v1alpha')
        )
        print("   Initialized v1alpha client for Veo.")
    except Exception as e:
        print(f"   ⚠️ Could not initialize v1alpha client ({e}), using default.")
        alpha_client = client

    model_name = "veo-3.1-generate-preview"
    print(f"   Attempting with model: {model_name}...")

    try:
        # Start the video generation operation
        operation = alpha_client.models.generate_videos(
            model=model_name,
            prompt=visual_prompt,
        )
        
        print("   Operation started. Waiting for video generation to complete...")
        
        # Poll the operation status until the video is ready.
        while not operation.done:
            print("   ...still processing...")
            time.sleep(10)
            operation = alpha_client.operations.get(operation)
            
        if operation.result:
             # Download the generated video.
            generated_video = operation.result.generated_videos[0]
            
            # Create unique filename with timestamp in videos folder
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            output_dir = "videos"
            os.makedirs(output_dir, exist_ok=True)
            output_file = os.path.join(output_dir, f"generated_video_{timestamp}.mp4")
            
            # Check if we have a URI to download from
            video_uri = getattr(generated_video.video, 'uri', None)
            
            if video_uri:
                # Append API key for authentication
                authenticated_uri = f"{video_uri}&key={api_key}"
                print(f"   Downloading video from authenticated URI...")
                
                try:
                    response = requests.get(authenticated_uri, stream=True)
                    response.raise_for_status()
                    
                    with open(output_file, "wb") as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                    print(f"✓ Video saved successfully to {output_file}")
                except Exception as download_err:
                    print(f"   ❌ Failed to download video from URI: {download_err}")
            else:
                print("   ⚠️ No URI found in generated video object. Trying fallback download...")
                # Fallback: try using the client's file download if URI is missing but name exists
                try:
                    video_content = alpha_client.files.content(file_name=generated_video.video.name)
                    with open(output_file, "wb") as f:
                        f.write(video_content)
                    print(f"✓ Video saved successfully to {output_file}")
                except Exception as e:
                     print(f"   ❌ Fallback download failed: {e}")
                     print(f"   Debug info - generated_video.video: {generated_video.video}")

        else:
            print("   ❌ Operation completed but no result returned.")

    except Exception as e:
        print(f"   ❌ Failed with {model_name}: {e}")


In [33]:
# Example Usage
user_request = "winter clothing for women"
script = generate_script_with_context(user_request)

print("\n" + "="*40 + "\nGENERATED SCRIPT\n" + "="*40 + "\n")
print(script)

🔍 Analyzing request: 'winter clothing for women'
   Querying Knowledge Graph...

   --- Relevant Knowledge Graph Items ---
   - [Entity] Cap-Toe Low-Heel Ankle Boot (Relevance: 0.24)
   - [Entity] Cap-Toe Knee-High Boot (Relevance: 0.22)
   - [Entity] Eleanor Loafer (Relevance: 0.22)
   - [Entity] Ballet Loafer (Relevance: 0.22)
   - [Entity] Cap-Toe Heeled Ankle Boot (Relevance: 0.21)
   - [Entity] Cap-Toe Crystal Mary Jane (Relevance: 0.18)
   - [Entity] apple (Relevance: 0.17)
   - [Entity] Penguin (Relevance: 0.17)
   - [Entity] Urban (Relevance: 0.16)
   - [Entity] israel (Relevance: 0.15)
   --------------------------------------

   Found 10 relevant items.
   Generating script with Gemini...

   --- Relevant Knowledge Graph Items ---
   - [Entity] Cap-Toe Low-Heel Ankle Boot (Relevance: 0.24)
   - [Entity] Cap-Toe Knee-High Boot (Relevance: 0.22)
   - [Entity] Eleanor Loafer (Relevance: 0.22)
   - [Entity] Ballet Loafer (Relevance: 0.22)
   - [Entity] Cap-Toe Heeled Ankle Boot 

In [34]:

# Run the video generation
create_video_from_script(script)

🎨 Extracting visual prompt from script...
   Visual Prompt: 'Bustling city street, light snow dusting sidewalks. Stylish woman walking confidently in cap-toe ankle boots. Quick cuts to modern architecture, quirky street art with animated penguins on building screens. Camera pans up to reveal urban-style coat and accessories. Close-up of boots, subtle shimmer effect. Woman continues walking, bites into a red apple, disappears into crowd.'

🎥 Generating video with Veo...
   Initialized v1alpha client for Veo.
   Attempting with model: veo-3.1-generate-preview...
   Visual Prompt: 'Bustling city street, light snow dusting sidewalks. Stylish woman walking confidently in cap-toe ankle boots. Quick cuts to modern architecture, quirky street art with animated penguins on building screens. Camera pans up to reveal urban-style coat and accessories. Close-up of boots, subtle shimmer effect. Woman continues walking, bites into a red apple, disappears into crowd.'

🎥 Generating video with Veo...
 

In [ ]:
# 📋 Listing available models to check support:
#  - models/embedding-gecko-001
#  - models/gemini-2.5-flash
#  - models/gemini-2.5-pro
#  - models/gemini-2.0-flash-exp
#  - models/gemini-2.0-flash
#  - models/gemini-2.0-flash-001
#  - models/gemini-2.0-flash-lite-001
#  - models/gemini-2.0-flash-lite
#  - models/gemini-2.0-flash-lite-preview-02-05
#  - models/gemini-2.0-flash-lite-preview
#  - models/gemini-exp-1206
#  - models/gemini-2.5-flash-preview-tts
#  - models/gemini-2.5-pro-preview-tts
#  - models/gemma-3-1b-it
#  - models/gemma-3-4b-it
#  - models/gemma-3-12b-it
#  - models/gemma-3-27b-it
#  - models/gemma-3n-e4b-it
#  - models/gemma-3n-e2b-it
#  - models/gemini-flash-latest
#  - models/gemini-flash-lite-latest
#  - models/gemini-pro-latest
#  - models/gemini-2.5-flash-lite
#  - models/gemini-2.5-flash-image-preview
#  - models/gemini-2.5-flash-image
#  - models/gemini-2.5-flash-preview-09-2025
#  - models/gemini-2.5-flash-lite-preview-09-2025
#  - models/gemini-3-pro-preview
#  - models/gemini-3-pro-image-preview
#  - models/nano-banana-pro-preview
#  - models/gemini-robotics-er-1.5-preview
#  - models/gemini-2.5-computer-use-preview-10-2025
#  - models/embedding-001
#  - models/text-embedding-004
#  - models/gemini-embedding-exp-03-07
#  - models/gemini-embedding-exp
#  - models/gemini-embedding-001
#  - models/aqa
#  - models/imagen-4.0-generate-preview-06-06
#  - models/imagen-4.0-ultra-generate-preview-06-06
#  - models/imagen-4.0-generate-001
#  - models/imagen-4.0-ultra-generate-001
#  - models/imagen-4.0-fast-generate-001
#  - models/veo-2.0-generate-001
#  - models/veo-3.0-generate-001
#  - models/veo-3.0-fast-generate-001
#  - models/veo-3.1-generate-preview
#  - models/veo-3.1-fast-generate-preview
#  - models/gemini-2.5-flash-native-audio-latest
#  - models/gemini-2.5-flash-native-audio-preview-09-2025
